### 목표
    - 타이타닉 데이터를 학습해서 생존자/사망자를 예측해보자
    - 머신러닝 전체 과정을 진행해보자

#### 1. 문제 정의
    - 생존과 사망을 예측
    - kaggle 경진대회 높은 순위를 차지해보자

#### 2. 데이터 수집(불러와서 확인)

In [156]:
# 환경
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


- q1. 타이타닉 train데이터와 test데이터를 변수 train, test에 저장하기(단, PassengerID 컬럼을 불러와서 인데스로 설정해서 불러오기)

In [157]:
test = pd.read_csv('./data/titanic/test.csv', index_col="PassengerId")
train = pd.read_csv('./data/titanic/train.csv', index_col="PassengerId")

train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [158]:
# 크래확인

print('훈련 데이터 크기',train.shape)
print('테스트 데이터 크기',test.shape)



훈련 데이터 크기 (891, 11)
테스트 데이터 크기 (418, 10)


### 3. 데이터 전처리 및 데이터 확보

In [159]:
# train - info()
train.info()
# 결측치 있는 컬럼 : Age, Cabin, embarked
# 데이터 타입 : 숫자(정수,실수), 문자열
# - 모델은 문제 데이터를 숫자인것만 이해할 수 있음 
# 문자 -> 숫자 : encoding(라벨인코딩, 원핫인코딩)

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [160]:
# test - info()
test.info()
# Survived 없음- 생존/사망
# y_test kagle이 알고 있음
# 경진대회 데이터는 테스트 문제까지만 제공
# 결측치 컬럼 o : Age, Fare, Cabin
# 데이터 타입 : 숫자(정수, 실수)


<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Ticket    418 non-null    object 
 7   Fare      417 non-null    float64
 8   Cabin     91 non-null     object 
 9   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


In [161]:
# 컬럼 정보 정리
train.columns

#'Survived' : 생존/사망 결과 값이 담긴 컬럼(0:사망, 1:생존)
# 'Pclass' : 승객 등급(1:상위층, 2:중상위층, 3:하위층)
# 'Name'    
# 'Sex'
# 'Age'
# 'SibSp'   : 형제자매/배우자 수(같이 탑승한)
# 'Parch'   : 부모/자식 수(같이 탑승한)
# 'Ticket'  : 티켓번호
# 'Fare'    : 요금
# 'Cabin'   : 객실번호
# 'Embarked': 승선항(S,C,Q)

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')

* Available features *
PassengerId - Passenger ID - Number  
Survived - Survived - Bool  
Pclass - Ticket class - available 1 = 1st, 2 = 2nd, 3 = 3rd - Number  
Name - First name, Last Name and optional Title - String  
Sex - Sex of passenger - available: male and female - String  
Age - Age in years - Number  
SibSp - of siblings / spouses aboard the Titanic - Number  
Parch - of parents / children aboard the Titanic - Number  
Ticket - Ticket Id - String  
Fare - Passenger fare / Ticker price - Number  
Cabin - Cabin number - String  
Embarked - Port of Embarkation - available C = Cherbourg, Q = Queenstown, S = Southampton - String  

- Train : Embarked 컬럼 결측치 채우기

In [162]:
# 데이터 확인 -> train 기준으로 확인
# 왜? 좋은 모델 만들기 위해서, 일반화된 모델을 만들기 위해서
# 학습시키는 데이터 -> train
train['Embarked'].value_counts()   # s 제일 많이 탐

# train['Embarked'] 결측치가 몇개 있는지? 2개
train['Embarked'].isnull().sum()
# train['Embarked'] r결측치가 있는 데이터 출력해보기
train[train['Embarked'].isnull()]


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [163]:
# 두사람의 데이터가 서로 비슷
# 같은 승선항에서 탔을 것이다 가정
# 사람이 가장 많이 탑승한 장소의 승선항값으로 채워보기
# 1. 하나씩 인덱싱 접근 train.loc[62,'Embark'] = 'S'
# fillna(값) : 비어 있다면 값으로 대체하는 함수
train['Embarked'].fillna("S" ,inplace=True)

train['Embarked'].isnull().sum()

0

 - test:Fare 컬럼 결측치 채우기


In [164]:
# test 데이터 Fare 인덱싱 후 결측치 존재한느 행 출력
# isnull(), 불리언 인덱싱

print(test[test["Fare"].isnull()])

# pclass에 연관되지 않았을까? -> 연관있음
# 성별에 연관되지 않았을까? -> 
# 상관계수 확인(iris 참고)
train.corr(numeric_only= True)  # 문자열로 된 컬럼 제외:numeric_only
# Fare컬럼과 연관이 있는 컬럼은 뭘까?

# -1 ~ 1 표현되는 상관계수
# -1, +1 가까울수록 상관이 어느정도 높음
# 0 가까울수록 상대적으로 상관관계가 덜함(적음)

             Pclass                Name   Sex   Age  SibSp  Parch Ticket  \
PassengerId                                                                
1044              3  Storey, Mr. Thomas  male  60.5      0      0   3701   

             Fare Cabin Embarked  
PassengerId                       
1044          NaN   NaN        S  


,Survived,Pclass,Age,SibSp,Parch,Fare
Survived,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [165]:
# train Fare 컬럼 기술통계량 확인하기
# describle()
# 중앙값, 평균값을 대표값을 선택할 건지 확인

train["Fare"].describe()    # 중앙값을 대표값으로 선택해보자

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [166]:
# Pclass, Sex -> Fare 중앙, 중앙, 평균 값으로 결측치를 대체해보자

# 1, female -> Fare 중앙값, 평균값을 확인하고 해당 값을 채워보자
# 1, male -> Fare 중앙값, 평균값
# 3, 남성 -> Fare 중앙값, 평균값을 확인해서 결측치 처리해야함
# test['Fare'] 3등급, 남성
# train 데이터를 기준으로 승객 등급, 성별 그룹핑(groupby)
group = train.loc[:,["Pclass","Fare","Sex"]].groupby(['Pclass','Sex']).median()
group

# test['Fare'] 결측치 채우기 7.92500
test['Fare'].fillna(7.92500,inplace=True)
test['Fare'].isnull().sum() # 처리확인
test[test['Fare'].isnull()] # 처리확인


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,


- Age 결측치 채우기
    - train, test 모두 처리
    - 나이의 평균, 중앙값으로 결측치를 채워보기
    - 다른 컬럼관 상관관계를 이용하여 결측치를 채워보기
    

In [167]:
# 기술통계량
# 중앙값 vs 평균값? 평균
train["Age"].describe()

test["Age"].describe()


count    332.000000
mean      30.272590
std       14.181209
min        0.170000
25%       21.000000
50%       27.000000
75%       39.000000
max       76.000000
Name: Age, dtype: float64

In [168]:
# Age 컬럼과 상관관계가 높은 컬럼 찾아보기

train.corr(numeric_only= True)["Age"].abs().sort_values(ascending=False)
test.corr(numeric_only= True)["Age"].abs().sort_values(ascending=False)


Age       1.000000
Pclass    0.492143
Fare      0.331969
SibSp     0.091587
Parch     0.061249
Name: Age, dtype: float64

In [169]:
# Pclass, Sex, Age 컬럼의 평균값 출력해보기
# 1 - female, age, 평균
# 1 - male, age 평균

gb1 = train.loc[:,["Pclass","Sex","Age"]].groupby(["Pclass","Sex"]).mean()
gb1

# gb1 3등급, male 나이 평균값 접근해보기
# 중복인덱스(멀티인덱스) 접근 방법 : gb1.loc[(3,"male")]
gb1.loc[(3,"male")]

# 1등급 여성 나이 평균 출력해보기
gb1.loc[(1,"female")]

Age    34.611765
Name: (1, female), dtype: float64

In [170]:
# train, test
train[train["Age"].isnull()]    # 결측치 177개

# 결측치 채우기위해서 각 승객의 등급, 성별을 확인 후 Age 평균값을 인덱싱하여 채움
# 판다스.apply(나이결측치처리기능)

def prepro_age(row):
   # 만약 나이가 결측치라면 값을 가져오고
   if np.isnan(row["Age"]): # 결측치확인한느 함수 np.isnan(대상)
       return gb1.loc[(row["Pclass"],row["Sex"])]
   # 만약 나이가 아니라면 원래 나이값을 그래돌 사용하자 
   else:
       return row["Age"]
    
# train.apply(prepro_age,axis=0)  #axis=0이면 컬럼 하나가 row에 처리하는 단위로 들어감    
train['Age'] = train.apply(prepro_age,axis=1)  #axis=1이면 한 사람의 대한 데이터가 row
train['Age'].isnull().sum()

# 1 한 사람에 대한 데이터가 row
# 0 컬럼 하나가 row에 처리하는 단위로 들어감
# test['Age'] 컬럼도 결측치 처리, train(gb1) 그대로 사용


0

In [171]:

test['Age'] = test.apply(prepro_age,axis=1)
test['Age'].isnull().sum()

0